In [1]:
!nvidia-smi

Mon Jul 22 21:08:40 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:30:00.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

<center>
  
# TABSYN: Tabular Data Synthesis with Diffusion Models

</center>

Introduction goes here.

# Imports and Setup

In [2]:
import os
import src
import json
import pandas as pd
from pprint import pprint

import torch
from torch.utils.data import DataLoader

from scripts.download_dataset import download_from_uci
from scripts.process_dataset import process_data
from src.data import preprocess, TabularDataset
from src.baselines.tabsyn.pipeline import TabSyn


NAME_URL_DICT_UCI = {
    "adult": "https://archive.ics.uci.edu/static/public/2/adult.zip",
    "default": "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip",
    "magic": "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip",
    "shoppers": "https://archive.ics.uci.edu/static/public/468/online+shoppers+purchasing+intention+dataset.zip",
    "beijing": "https://archive.ics.uci.edu/static/public/381/beijing+pm2+5+data.zip",
    "news": "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip",
}

DATA_DIR = "/projects/aieng/diffusion_bootcamp/data/tabular_copy"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
DATA_NAME = "adult"

MODEL_PATH = "/projects/aieng/diffusion_bootcamp/models/tabular/tabsyn_copy"

# Adult Dataset

For more detailed explaination, please refer to the TabDDPM reference implementation.

In [3]:
# download data
download_from_uci(DATA_NAME, RAW_DATA_DIR, NAME_URL_DICT_UCI)

# process data
INFO_DIR = "data_info"
process_data(DATA_NAME, INFO_DIR, DATA_DIR)

# review data
df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "train.csv"))
df.head(5)

Start processing dataset adult from UCI.
Aready downloaded.
adult (32561, 15) (16281, 15) (32561, 15)
Numerical (32561, 6)
Categorical (32561, 8)
Processing and Saving adult Successfully!
adult
Total 48842
Train 32561
Test 16281
Num 6
Cat 9


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [4]:
# clean data
value = " ?"
if value in df.values:
    print(f"{value} exists in the DataFrame.")
else:
    print(f"{value} does not exist in the DataFrame.")

# review json file and its contents
with open(f"{PROCESSED_DATA_DIR}/{DATA_NAME}/info.json", "r") as file:
    data_info = json.load(file)
pprint(data_info)

 ? exists in the DataFrame.
{'cat_col_idx': [1, 3, 5, 6, 7, 8, 9, 13],
 'column_info': {'0': {},
                 '1': {},
                 '10': {},
                 '11': {},
                 '12': {},
                 '13': {},
                 '14': {},
                 '2': {},
                 '3': {},
                 '4': {},
                 '5': {},
                 '6': {},
                 '7': {},
                 '8': {},
                 '9': {},
                 'categorizes': [' >50K', ' <=50K'],
                 'max': 99.0,
                 'min': 1.0,
                 'type': 'categorical'},
 'column_names': ['age',
                  'workclass',
                  'fnlwgt',
                  'education',
                  'education.num',
                  'marital.status',
                  'occupation',
                  'relationship',
                  'race',
                  'sex',
                  'capital.gain',
                  'capital.loss',
          

# TabSyn Algorithem

Explanation of the algorithm goes here.

<p align="center">
<img src="figures/tabsyn.jpg" width="1000"/>
</p>

## Load Config

In [5]:
config_path = os.path.join("src/baselines/tabsyn/configs", f"{DATA_NAME}.toml")
raw_config = src.load_config(config_path)

pprint(raw_config)

{'loss_params': {'lambd': 0.7, 'max_beta': 0.01, 'min_beta': 1e-05},
 'model_params': {'d_token': 4, 'factor': 32, 'n_head': 1, 'num_layers': 2},
 'sample': {'steps': 50},
 'task_type': 'binclass',
 'train': {'diffusion': {'batch_size': 4096,
                         'num_dataset_workers': 4,
                         'num_epochs': 10001},
           'optim': {'diffusion': {'factor': 0.9,
                                   'lr': 0.001,
                                   'patience': 20,
                                   'weight_decay': 0},
                     'vae': {'factor': 0.95,
                             'lr': 0.001,
                             'patience': 10,
                             'weight_decay': 0}},
           'vae': {'batch_size': 4096,
                   'num_dataset_workers': 4,
                   'num_epochs': 4000}},
 'transforms': {'cat_encoding': None,
                'cat_min_frequency': None,
                'cat_nan_policy': None,
                'normalizat

## Make Dataset

In [6]:
# preprocess data
X_num, X_cat, categories, d_numerical = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                   transforms = raw_config["transforms"],
                                                   task_type = raw_config["task_type"])

# separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# convert to float tensor
X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# create dataset module
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size = raw_config["train"]["vae"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["vae"]["num_dataset_workers"],
)

data_path /projects/aieng/diffusion_bootcamp/data/tabular_copy/processed_data/adult
No NaNs in numerical features, skipping


In [7]:
print("train dataset num shape: ", train_data.X_num.shape)
print("test dataset num shape: ", X_test_num.shape)

print("train dataset cat shape: ", train_data.X_cat.shape)
print("test dataset cat shape: ", X_test_cat.shape)

train dataset num shape:  torch.Size([32561, 6])
test dataset num shape:  torch.Size([16281, 6])
train dataset cat shape:  torch.Size([32561, 9])
test dataset cat shape:  torch.Size([16281, 9])


## Instantiate Model

In [8]:
tabsyn = TabSyn(train_loader,
                X_test_num, X_test_cat,
                num_numerical_features = d_numerical,
                num_classes = categories,
                device = device)

## Train Model
### First: Train VAE

In [9]:
# laod and prepare VAE model for training
tabsyn.load_vae_model(**raw_config["model_params"], optim_params = raw_config["train"]["optim"]["vae"])

self.category_embeddings.weight.shape=torch.Size([104, 4])
self.category_embeddings.weight.shape=torch.Size([104, 4])
Successfully loaded VAE model.


In [ ]:
tabsyn.train_vae(**raw_config["loss_params"],
                 num_epochs = raw_config["train"]["vae"]["num_epochs"],
                 model_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "model.pt"),
                 encoder_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "encoder.pt"),
                 decoder_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "decoder.pt"))

In [ ]:
# embed all inputs in the latent space
tabsyn.save_vae_embeddings(X_train_num, X_train_cat,
                           vae_ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Successfully saved pretrained embeddings on disk!


### Second: Train Diffusion Model

In [ ]:
# load latent space embeddings
train_z, _ = tabsyn.load_vae_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))  # train_z dim: B x in_dim

# normalize embeddings
mean, std = train_z.mean(0), train_z.std(0)
train_z = (train_z - mean) / 2
latent_train_data = train_z

# create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size = raw_config["train"]["diffusion"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["diffusion"]["num_dataset_workers"],
)

In [ ]:
# load and prepare diffusion model for training
tabsyn.load_diffusion_model(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = raw_config["train"]["optim"]["diffusion"])

MLPDiffusion(
  (proj): Linear(in_features=60, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=60, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10616892
Successfully loaded diffusion model.


In [ ]:
# train diffusion model
tabsyn.train_diffusion(latent_train_loader,
                       num_epochs = raw_config["train"]["diffusion"]["num_epochs"],
                       ckpt_path = os.path.join(MODEL_PATH, DATA_NAME))

Epoch 9/9: 100%|██████████| 8/8 [00:01<00:00,  6.18it/s, Loss=0.397]


Time:  12.037148237228394


## Load pretrained Model

In [ ]:
# import importlib
# import src.baselines.tabsyn.pipeline as pipeline
# importlib.reload(pipeline)
# TabSyn = getattr(pipeline, "TabSyn")

In [ ]:
# laod VAE model
tabsyn.load_vae_model(**raw_config["model_params"], optim_params = None)

# load latent embeddings of input data
train_z, token_dim = tabsyn.load_vae_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))

# load diffusion model
tabsyn.load_diffusion_model(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = None)

# load state from checkpoint
tabsyn.load_model_state(ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME),
                        dif_ckpt_name = "model.pt")

self.category_embeddings.weight.shape=torch.Size([104, 4])
self.category_embeddings.weight.shape=torch.Size([104, 4])
Successfully loaded VAE model.
MLPDiffusion(
  (proj): Linear(in_features=60, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=60, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10616892
Successfully loaded diffusion model.
Loaded model state from /projects/aieng/diffusion_bootcamp/models/tabular/tabsyn_copy/adult


## Sample Data

In [ ]:
# load data info file
with open(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "info.json"), "r") as file:
    data_info = json.load(file)
data_info["token_dim"] = token_dim

# get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                                     transforms = raw_config["transforms"],
                                                                     task_type = raw_config["task_type"],
                                                                     inverse = True)

# sample data
tabsyn.sample(train_z,
              info = data_info,
              num_inverse = num_inverse,
              cat_inverse = cat_inverse,
              save_path = os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

data_path /projects/aieng/diffusion_bootcamp/data/tabular_copy/processed_data/adult
No NaNs in numerical features, skipping
(32561, 9)
Time: 15.157551765441895
Saving sampled data to /projects/aieng/diffusion_bootcamp/data/tabular_copy/synthetic_data/adult/tabsyn.csv


## Synthetic Data

Finally here, we review the synthesized data. In the following `evaluate_synthetic_data.ipynb` notebook, we will evaluate this synthesized data with respect to various metrics.

In [ ]:
df = pd.read_csv(os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

# Display the first few rows of the DataFrame
df.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,35.000000,Private,154501.34,HS-grad,6.0,Never-married,Exec-managerial,Wife,Black,Male,0.0,0.0,40.0,Portugal,<=50K
1,36.000000,State-gov,254158.64,Prof-school,9.0,Married-civ-spouse,Adm-clerical,Husband,Black,Male,0.0,0.0,40.0,Portugal,<=50K
2,33.141235,State-gov,187729.34,Assoc-acdm,9.0,Never-married,Exec-managerial,Wife,Black,Male,0.0,0.0,35.0,Portugal,<=50K
3,49.000000,State-gov,165583.39,HS-grad,10.0,Married-civ-spouse,Adm-clerical,Wife,Other,Male,0.0,0.0,40.0,Portugal,>50K
4,44.000000,State-gov,187748.36,Assoc-acdm,9.0,Never-married,Adm-clerical,Wife,Black,Male,0.0,0.0,40.0,Portugal,<=50K


## References

**Zhang, Hengrui, et al.** "Mixed-type tabular data synthesis with score-based diffusion in latent space." *International Conference on Learning Representations (ICLR)* (2023).

**GitHub Repository:** [Amazon Science - Tabsyn](https://github.com/amazon-science/tabsyn)